In [90]:
import NutritionCalculator as NC
import pandas as pd
import glob
import pandas as pd
import numpy as np
from scipy.optimize import minimize
#################사용자 정보 입력#################
age=60
gender='Female'
height=165/100
weight=51
state='CKD' # NA (일반인), CKD (만성신부전), HD (혈액투석), PD (복막투석)
PDk=0 # 복막투석 시 투석액 칼로리
eGFR=50 # 사구체 여과율
uACR = 50 
protein_loss=0
albumin=3.4 # 혈중 알부민
FBG=90
OGTT=135
HbA1c=7.1
TG=600
LDL=105
SBP=130
DBP=80
CK=6
CP=5
CCa=4
mealtype = "식단" # 식단 메인반찬 단품요리 / 간식 간식콤보 음료
foodtype='음식' #가공식품 음식

##################################################
top_n=10

Ncalc = NC.NutritionCalculator(gender, height, weight, state, PDk, eGFR, uACR, protein_loss, albumin, FBG, OGTT, HbA1c, TG, LDL, SBP, DBP, CK, CP, CCa)

In [91]:
def import_diet(mealtype, foodtype):
    EER, _, _, _, _, _, _, _, _, _, _ = Ncalc.calculate_meal_nutrient_criteria(mealtype)
    resulting_diets = []
    total_nutritions = []

    if mealtype == '식단':
        name = 'diet'
    elif mealtype == '메인반찬':
        name = 'main'
    elif mealtype == '단품요리':
        name = 'one' 
    elif mealtype == '간식콤보':
        name = 'snack_combi'
    elif mealtype == '음료':
        name = 'beverage'
    else:
        name = 'snack'
    a='d' if foodtype == '음식' else 'p'
    
    file_list = glob.glob(f'food_DB/processed_DB/{name}/{name}{a}*.txt')
    for file in file_list[:1]:
        with open(file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            i = 0
            while i < len(lines)-2:
                diet = eval("["+lines[i].strip().split(": ")[1].replace("(","").replace(")","")+"]")
                nutritions= lines[i+1]

                resulting_diets.append(diet)
                total_nutritions.append(nutritions)
                i += 3
    return resulting_diets, total_nutritions

In [17]:
def diet_scoring(total_nutrition, mealtype):
    EER, CHOA, CHOB, ProA, ProB, FatA, FatB, Na, K, P, Sugar = Ncalc.calculate_meal_nutrient_criteria(mealtype)
    df = pd.DataFrame(Ncalc.calculate_day_nutrient_criteria())
    w=df['Weight'][0]
    
    Naw = 1.5 if 'Na' in w else 1
    Sugarw = 1.5 if 'Sugar' in w else 1
    if eGFR < 30 or uACR>300 or state != 'X':
        Kw = 1.5 if 'K' in w else 1
        Pw = 1.5 if 'P' in w else 1
    else:
        Kw = 0
        Pw = 0

    if total_nutrition[1]<CHOA:
        CHOS = max(1-((total_nutrition[1]-CHOA)/CHOA)**2, -1) *5
    elif total_nutrition[1]>CHOB:
        CHOS = max(1-((total_nutrition[1]-CHOB)/CHOA)**2, -1) *5
    else:
        CHOS = 1

    if total_nutrition[2]<ProA:
        ProS = max(1-((total_nutrition[2]-ProA)/ProA)**2, -1) *5
    elif total_nutrition[2]>ProB:
        ProS = max(1-((total_nutrition[2]-ProB)/ProA)**2, -1) *5
    else:
        ProS = 1

    if total_nutrition[3]<FatA:
        FatS = max(1-((total_nutrition[3]-FatA)/FatA)**2, -1) *5
    elif total_nutrition[3]>FatB:
        FatS = max(1-((total_nutrition[3]-FatB)/FatA)**2, -1) *5
    else:
        FatS = 1
    
    NaS = max(-total_nutrition[4]*Naw/Na, -2) *5
    SugarS = max(-total_nutrition[5]*Sugarw/Sugar, -2) *5
    KS = max(-total_nutrition[6]*Kw/K,-2) *5
    PS = max(-total_nutrition[7]*Pw/P, -2) *5
        
    Total_score = 75+CHOS+ProS+FatS+NaS+SugarS+KS+PS
    
    return CHOS,ProS,FatS,NaS,SugarS,KS,PS,Total_score

In [76]:
def optimize_size(food, total_nutrition):
    nutrition_data = pd.DataFrame(eval(total_nutrition), columns=['에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '나트륨(mg)', '당류(g)', '칼륨(mg)', '인(mg)'])
    def objective_function(weights, nutrition_data):
        modified_nutrition = nutrition_data.copy()
        for i, weight in enumerate(weights):
            modified_nutrition.iloc[i] *= weight
        total_nutrition = modified_nutrition.fillna(0).sum(axis=0).values.tolist()
        return diet_scoring(total_nutrition, mealtype)[-1]

    initial_guess = np.random.uniform(0.5, 1.5, 5)
    bounds = [(0.5, 1.5)] * 5
    optimal_weights = minimize(objective_function, initial_guess, args=(nutrition_data,), bounds=bounds).x

    optimalized_nutrition_data = nutrition_data.copy()
    for i, weight in enumerate(optimal_weights):
        optimalized_nutrition_data.iloc[i] *= weight
    summed_nutrition = optimalized_nutrition_data.fillna(0).sum(axis=0)
    CHOS,ProS,FatS,NaS,SugarS,KS,PS,diet_score = diet_scoring(summed_nutrition, mealtype)
    
    total_nutritionss = summed_nutrition.values.tolist()

    for j in range(len(optimal_weights)):
        food[j][0] = str(round(float(food[j][0]) * optimal_weights[j], 1))
    
    return food, total_nutritionss, CHOS,ProS,FatS,NaS,SugarS,KS,PS,diet_score

In [86]:
def top_diets():
    resulting_diets, total_nutritions = import_diet(mealtype, foodtype) # type: ignore
    diet_scores = []
    for i in range(len(resulting_diets)):
        total_nutrition = total_nutritions[i]
        resulting_diet = resulting_diets[i]
            
        food, nutritions, CHOS,ProS,FatS,NaS,SugarS,KS,PS,diet_score = optimize_size(resulting_diet, total_nutrition)
        diet_scores.append((food, nutritions[0],nutritions[1],nutritions[2],nutritions[3],nutritions[4],nutritions[5], nutritions[6],nutritions[7],
                            CHOS,ProS,FatS,NaS,SugarS,KS,PS,diet_score))

    top_diets = sorted(diet_scores, key=lambda x: x[16], reverse=True)
    return top_diets

def diet_ranking(top_diet):
    top_diets = top_diet[:top_n]
    results_list = []
    for rank, (diet, Calorie, CHO, Pro, Fat, Na, K, P, Sugar, CHOS,ProS,FatS,NaS,SugarS,KS,PS, score) in enumerate(top_diets, start=1):
        if mealtype == '메인반찬':
            comp = str(diet[2]+' '+diet[0]+'g')
        elif mealtype == '단품요리' or mealtype == '간식' or mealtype == '음료':
            diett = diet[0]
            comp = str(diett[2]+' '+diett[0]+'g')
        else:
            comp = ' & '.join(map(str, list(str(name[2]+' '+name[0]+'g') for name in diet)))
        result_dict = {
            "순위": rank,
            "식단 구성": comp,
            "식단 점수": round(score, 2),
            "칼로리": f"{round(Calorie,2)}kcal",
            "탄수화물": f"{round(CHO,2)}g ({round(CHOS,2)})",
            "단백질" : f"{round(Pro,2)}g ({round(ProS,2)})",
            "지방" : f"{round(Fat,2)}g ({round(FatS,2)})",
            "나트륨" : f"{round(Na,2)}mg ({round(NaS,2)})",
            "칼륨" : f"{round(K,2)}mg ({round(KS,2)})",
            "인" : f"{round(P,2)}mg ({round(PS,2)})",
            "당류" : f"{round(Sugar,2)}g ({round(SugarS,2)})"
        }
        results_list.append(result_dict)

    result_df = pd.DataFrame(results_list)
    result_df = result_df.set_index('순위')
    
    return result_df

def find_food():
    all_food = []
    for mealtype in ['메인반찬','단품요리', '간식', '음료']:
        for foodtype in ['음식','가공식품']:
            resulting_diets, total_nutritions = import_diet(mealtype, foodtype) # type: ignore
            diet_scores = []
            for i in range(len(resulting_diets)):
                total_nutrition = total_nutritions[i]
                resulting_diet = resulting_diets[i]
                if mealtype == '메인반찬':
                    resulting_diet = resulting_diets[i][0]
                CHOS,ProS,FatS,NaS,SugarS,KS,PS,diet_score = diet_scoring(total_nutrition, mealtype)
                diet_scores.append((resulting_diet, total_nutrition[0],total_nutrition[1],total_nutrition[2],total_nutrition[3],
                                    total_nutrition[4],total_nutrition[5], total_nutrition[6],total_nutrition[7],diet_score))
                

In [92]:
diet_ranking(top_diets())

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30704\2154535271.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if total_nutrition[1]<CHOA:
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30704\2154535271.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif total_nutrition[1]>CHOB:
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30704\2154535271.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

,식단 구성,식단 점수,칼로리,탄수화물,단백질,지방,나트륨,칼륨,인,당류
순위,,,,,,,,,,
1,기장밥 300.0g & 갈비구이_소고기 150.0g & 김구이_조미김 7.5g & ...,37.41,1834.95kcal,53.17g (4.97),207.93g (-5),87.98g (-5),5332.23mg (-10),4.38mg (-10),2619.15mg (-10),1363.22g (-2.55)
2,기장밥 300.0g & 갈비구이_소고기 450.0g & 김구이_조미김 7.5g & ...,34.40,1927.01kcal,68.33g (1),209.47g (-5),90.83g (-5),4406.11mg (-10),2.75mg (-10),2640.11mg (-10),1382.85g (-1.6)
3,기장밥 300.0g & 갈비구이_소고기 450.0g & 김구이_조미김 7.5g & ...,34.37,1943.02kcal,73.9g (1),209.84g (-5),89.92g (-5),4676.87mg (-10),2.8mg (-10),2558.34mg (-10),1379.57g (-1.63)
4,기장밥 242.1g & 갈비구이_소고기 170.7g & 김구이_조미김 7.3g & ...,34.31,1732.99kcal,77.18g (1),184.97g (-5),76.19g (-5),3737.66mg (-10),2.9mg (-10),2298.75mg (-10),1221.2g (-1.69)
5,기장밥 300.0g & 갈비구이_소고기 450.0g & 김구이_조미김 7.5g & ...,33.87,2185.2kcal,126.84g (1.32),215.14g (-5),90.95g (-5),4718.77mg (-10),4.2mg (-10),2672.1mg (-10),1430.78g (-2.45)
6,기장밥 300.0g & 갈비구이_소고기 450.0g & 김구이_조미김 7.5g & ...,33.12,2200.35kcal,129.58g (0.9),216.13g (-5),90.99g (-5),4602.07mg (-10),4.78mg (-10),2851.5mg (-10),1464.68g (-2.79)
7,기장밥 300.0g & 갈비구이_소고기 450.0g & 김구이_조미김 7.5g & ...,32.96,2195.7kcal,128.43g (1.08),215.98g (-5),91.02g (-5),4683.52mg (-10),5.35mg (-10),2786.1mg (-10),1436.78g (-3.12)
8,기장밥 300.0g & 갈비구이_소고기 450.0g & 김구이_조미김 7.5g & ...,32.90,1898.69kcal,62.07g (1),209.7g (-5),90.38g (-5),4803.92mg (-10),5.32mg (-10),2701.43mg (-10),1393.03g (-3.1)
9,기장밥 300.0g & 갈비구이_소고기 450.0g & 김구이_조미김 7.5g & ...,32.74,1941.6kcal,72.03g (1),210.58g (-5),90.25g (-5),4564.89mg (-10),5.59mg (-10),2611.62mg (-10),1389.38g (-3.26)
